# Tutorial: Answering Multihop Questions with Agents

- **Level**: Intermediate
- **Time to complete**: 10 minutes
- **Nodes Used**: `Agent`, `PromptNode`, `InMemoryDocumentStore`, `FARMReader` and `ExtractiveQAPipeline`
- **Goal**: After completing this tutorial, you will have learned how to use Agents to build a multi-hop question answering system with an `ExtractiveQAPipeline` as a tool
- **Prerequisites**: An [OpenAI API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key)


## Overview

The Agent class uses a large language model (LLM) to make decisions and come up with the best next course of action. You can provide the `Agent` with a set of `Tools` that it can choose to use to reach a result. At each iteration, the agent will pick a tool from the ones available to it. Based on the result, the Agent has two options: It will either decide to select a tool again and do another iteration, or it will decide that it has reached a conclusion and return the final answer.

In this tutorial, we will provide the Agent with just one tool to answet questions: a commonly used Haystack component, the `ExtractiveQAPipeline`.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`. In this tutorial, we'll use a [Hugging Face dataset](https://huggingface.co/datasets/Tuana/presidents) that has already been prepared as Haystack `Documents`, so we will install `datasets` too:

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]
pip install datasets


### Enabling Telemetry 
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(23)

Set the logging level to INFO:

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Create an Extractive QA Pipeline

Now, we will introduce an `ExtractiveQAPipeline` as a `Tool` to our `Agent`. To do so, we'll first write our documents about the presidents of the USA into a `DocumentStore` and then create our pipeline.

### 1) Write documents to the DocumentStore

In [ ]:
from haystack.document_stores import InMemoryDocumentStore
from datasets import load_dataset

remote_dataset = load_dataset('Tuana/presidents', split="train")

document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(remote_dataset)

### 2) Create an ExtractiveQAPipeline:
Let's define our retriever and reader to to use in an `ExtractiveQAPipeline`:

In [ ]:
from haystack.nodes import BM25Retriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

retriever = BM25Retriever(document_store=document_store)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
presidents_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)


### 3) Let's test the pipeline!

Now that you have an `ExtractiveQAPipeline`, go ahead and ask it a question about the presidents of the USA. 

Below, we're asking the question: "Who was the wife of 44th president of the USA?"

Notice how this is 2 questions in one. An extractive model will struggle to find the answer to this question unless the answer is phrased clearly in our documents. For example: "The wife of the 44th president is Michelle Obama".

On the other hand, it does well with a question such as "Who was the 44th president of the USA?"

In [ ]:
from haystack.utils import print_answers

#result = presidents_qa.run("Who was the 44th president of the USA?")
result = presidents_qa.run("Who is the wife of the 44th president of the USA?")

print_answers(result, "minimum")

## Create an Agent with the `ExtractiveQAPipeline` as a `Tool`
### 1) To create the Agent, we'll make use of an Open AI model. So first, provide your Open AI key:

In [ ]:
from getpass import getpass

api_key_prompt = "Enter OpenAI API key:" 
api_key = getpass(api_key_prompt)

### 2) Initialize the Agent 

The `Agent` needs to determine the next best course of action at each iteration. It does this by using an LLM, and a prompt designed specially for this use case. Our `Agent` does this by using a `PromptNode` which by default uses the ["zero-shot-react" `PromptTemplate` ](https://github.com/deepset-ai/haystack/blob/444a3116c42d2c8852d27aa8093ac92c8e85ab88/haystack/nodes/prompt/prompt_node.py#L337). 

Here, let's define an `Agent` that uses the `text-davinci-003` model by OpenAI.

In [ ]:
from haystack.agents import Agent
from haystack.nodes import PromptModel, PromptNode

prompt_model = PromptModel(model_name_or_path="text-davinci-003", api_key=api_key)
prompt_node = PromptNode(model_name_or_path=prompt_model, stop_words=["Observation:"])
agent = Agent(prompt_node=prompt_node)

### 3) Provide the Agent with a Tool
Next, let's add our `ExtractiveQAPipeline` into the Agent's arsenal. The Agent will then be able to use this pipeline when it decides it could be useful.

To do so, let's define a tool and make sure to give it a description. The exact wording of your description matters a lot here. The agent uses it to understand in which cases it should pick this tool. If the agent fails to pick the right tool, adjusting the description might help.

In [ ]:
from haystack.agents import Tool

search_tool = Tool(name="USA_Presidents_QA",pipeline_or_node=presidents_qa,
                  description="useful for when you need to answer questions related to the presidents of the USA.",
              )
agent.add_tool(search_tool)

### 4) Ask a question!


In [ ]:
result = agent.run("Who is the wife of the 44th president?")

print(result["transcript"].split('---')[1])

Congratulations! 🎉 You've used an Agent that can use an extractive model iteratively, to arrive at a final answer to a multi-hop question!!!

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Discord](https://haystack.deepset.ai/community) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)
